In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'a') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [3]:
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

In [4]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [5]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [6]:
def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [7]:
SYSTEM_PROMPT_8="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":{
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  },
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [8]:
SELF_REF="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":{
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  },
}"""

In [9]:
use_case_directory_path = r"C:\Users\congc\Desktop\GPT\Dissertation-GPT\dataset\SpecificationData\Shopee(new)"
usecase_name_list = []
project_name = "Online Shopping System"
for filename in os.listdir(use_case_directory_path):
    usecase_name_list.append (filename.split(".txt")[0])
print(usecase_name_list)

test_scenario_directory_path = r"C:\Users\congc\Desktop\GPT\Dissertation-GPT\evaluate\bestScenario\Shopee(new)"
save_testcase_path = r"C:\Users\congc\Desktop\GPT\Dissertation-GPT\evaluate\bestScenario\bestTCs\Shopee(new)"
save_testcase_raw_path = r"C:\Users\congc\Desktop\GPT\Dissertation-GPT\evaluate\bestScenario\bestTCs\Shopee(new)\raw"

['add to cart', 'checkout', 'product information display', 'purchase', 'search']


In [10]:

for usecase in usecase_name_list:
  base_name = os.path.basename(usecase)
  usecase_name =usecase
  usecase_path = os.path.join (use_case_directory_path,f"{usecase_name}.txt" )
  testscenario_path = os.path.join (test_scenario_directory_path,f"{usecase_name}.txt" )
  testscenario_list = read_file_to_list(testscenario_path)
  print(testscenario_path)
  print(testscenario_list)
  for testscenario in testscenario_list:
    content = "Test scenarios:"+ testscenario+  "\nUse case:" + read_file_content(usecase_path)
    print(content)
    testscenario = testscenario.replace('"','').replace(':','').replace('?','').replace('*','').replace('/','')
    testcase_path = os.path.join (save_testcase_path,f"{usecase_name}-{testscenario}.txt" )
    testcase_raw_path = os.path.join (save_testcase_raw_path,f"{usecase_name}-{testscenario}.txt" )
    promptTestCase = [
    { "role": "system", "content": SYSTEM_PROMPT_8},
    { "role": "user", "content": content}
    ]
    gpt_response = askJSON(promptTestCase, client, model)
    json_data = json.loads(gpt_response)
    write_json_to_txt_raw(json_data, testcase_raw_path)

    promptSelfRef = [
        { "role": "system", "content": SELF_REF},
        { "role": "user", "content": "Test scenario: "+ testscenario + "\n" + gpt_response}
    ]
    final_response = askJSON(promptSelfRef, client, model)
    json_fin = json.loads(final_response)
    write_json_to_txt(json_fin, testcase_path)
  print("f{usecase_name} gen test case complete")

C:\Users\congc\Desktop\GPT\Dissertation-GPT\evaluate\bestScenario\Shopee(new)\add to cart.txt
['1. Successful Product Addition to Cart', '2. Product Out of Stock Scenario', '3. User chooses a product option before adding to cart', '4. User adjusts quantity before adding product to cart', '5. User tries to add a product with multiple options without selecting an option', '6. User tries to add a quantity of a product that exceeds the current stock', '7. User tries to decrease the quantity of a product to less than one', '9. User tries to add an out-of-stock option of a product with multiple options']
Test scenarios:1. Successful Product Addition to Cart
Use case:Use case name: Add item to cart
Description: This feature allows users to add items to their shopping cart for future purchase.
Actor: User
Preconditions: User is logged into their Shopee account.
Postconditions: 
The item is successfully added to the user's shopping cart.
The user can view and manage items in their shopping cart